In [ ]:
import asyncio
from twikit import Client, TooManyRequests
from configparser import ConfigParser
import pandas as pd
from datetime import datetime, timedelta

# Constantes
MINIMUMS = 100  
REQUETE = 'iphone 16 | nouvel-iphone -Fr'  
# Lire les informations de connexion depuis le fichier config.ini
config = ConfigParser()
config.read('config.ini')
username = config['X']['username']
email = config['X']['email']
password = config['X']['password']

client = Client(language='fr-FR')  # Configurer pour obtenir des résultats en français

# Fonction principale async
async def main():
    # Authentification
    await client.login(auth_info_1=username, auth_info_2=email, password=password)
    
    # Sauvegarder les cookies après l'authentification
    client.save_cookies('cookies.json')
    client.load_cookies('cookies.json')  # Charger les cookies pour les sessions futures

    # Initialisation de la liste pour les tweets et compteur
    donne = []
    tweet_count = 0

    # Rechercher les tweets avec la requête spécifiée
    tweets = await client.search_tweet(REQUETE, product='Top')

    # Boucle pour paginer les tweets
    while tweet_count < MINIMUMS:
            # Récupérer la page suivante de tweets
            encore = await tweets.next()

            for tweet in encore:
                post_data = {
                    'Texte': tweet.full_text,
                    'Nombre_Likes': tweet.favorite_count,
                    'Nombre_Retweets': tweet.retweet_count,
                    'Followers_Utilisateur': tweet.user.followers_count,
                    'Utilisateur': tweet.user.screen_name,
                    'Date_Publication': tweet.created_at
                }
                donne.append(post_data)
                tweet_count += 1

            
                if tweet_count >= MINIMUMS:
                    break
           
        

    if donne:
        df = pd.DataFrame(donne)
        df.to_csv('tweets_data.csv', index=False)
        print(f"{len(df)} tweets collectés et sauvegardés dans 'tweets_data.csv'")
    else:
        print("Aucune donnée collectée.")

# Exécuter la fonction async
await main()


100 tweets collectés et sauvegardés dans 'tweets_data.csv'
